# Hybrid Stock Prediction Model Training
In the "HybridStockPredictionModel" notebook we created our model that can be used to make efficient stock prediction for new business ideas.

At first we create our Dataset class that will be used to train the model:


# Load Libraries and Set Up Dependencies

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sentence_transformers import SentenceTransformer
import torch.nn.functional as F
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Load and Preprocess the Dataset

In [8]:
# Load the dataset
df = pd.read_csv("../Dataset/Data/real_company_stock_dataset.csv")

# Define columns
idea_column = "business_description"
static_feature_columns = ["market_size", "investment", "team_strength"]
historical_columns = [col for col in df.columns if col.startswith("month_")]

# Define the target as stock performance for the next 6 months
target_columns = historical_columns[-6:]  # Last 6 months of performance
forecast_steps = 6

# Prepare your features and target
ideas = df[idea_column].values
static_features = df[static_feature_columns].values
historical_data = df[historical_columns].values
targets = df[target_columns].values

# Scale static features
scaler_static = StandardScaler()
static_features = scaler_static.fit_transform(static_features)

print(static_features)

# Train-test split
ideas_train, ideas_val, static_train, static_val, hist_train, hist_val, y_train, y_val = train_test_split(
    ideas, static_features, historical_data, targets, test_size=0.2, random_state=42
)


# Convert the ideas to embeddings using SentenceTransformer
from sentence_transformers import SentenceTransformer

# Load the pre-trained model (this will map each idea to a vector of length 384)
text_encoder = SentenceTransformer('all-MiniLM-L6-v2')

# Convert the text ideas into embeddings (numerical vectors)
ideas_train_embeddings = text_encoder.encode(ideas_train, convert_to_numpy=True)
ideas_val_embeddings = text_encoder.encode(ideas_val, convert_to_numpy=True)

# Now convert these embeddings into torch tensors
ideas_train_tensor = torch.tensor(ideas_train_embeddings, dtype=torch.float32)
ideas_val_tensor = torch.tensor(ideas_val_embeddings, dtype=torch.float32)

# Convert static features and target variables into torch tensors
static_train_tensor = torch.tensor(static_train, dtype=torch.float32)
static_val_tensor = torch.tensor(static_val, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

[[-5.03179349e-02 -9.10303363e-02  2.81726804e-02]
 [ 1.08708214e-02 -5.77960233e-02 -1.08341395e-02]
 [-1.59124331e-01 -1.92085975e-01 -1.44473558e-01]
 [-1.62370808e-01 -1.95187153e-01 -1.49893453e-01]
 [ 7.05983650e-01  4.21348319e-01  1.22711915e+00]
 [-1.63057247e-01 -1.96234387e-01 -1.50304051e-01]
 [            nan -1.92888176e-01 -1.48856692e-01]
 [-1.27554504e-01 -1.78588093e-01 -5.73241099e-02]
 [-1.59189716e-01 -1.92671428e-01 -1.28378112e-01]
 [-1.43540799e-01 -1.87260564e-01 -1.10855838e-01]
 [ 2.29934858e-02 -7.62029926e-02  2.60160610e-01]
 [ 6.27266284e+00  3.37953530e+00  1.53301473e+00]
 [-1.55511183e-01 -1.67241152e-01 -1.48097086e-01]
 [-1.57837985e-01 -1.76129185e-01 -1.02120363e-01]
 [ 7.31163531e-01  1.12339379e+00  3.62810136e-01]
 [-1.45614584e-01  5.04574910e-02 -1.22383380e-01]
 [-1.62145818e-01 -1.81904490e-01 -1.44422233e-01]
 [-1.62714011e-01 -1.95857532e-01 -1.49575239e-01]
 [ 1.14949351e+00  1.10314457e+00  2.90955467e-01]
 [ 8.08429126e-02 -9.69443196e-

# Define the PyTorch Dataset

In [3]:
class StockDataset(Dataset):
    def __init__(self, ideas, static_features, historical_data, targets):
        self.ideas = ideas
        self.static_features = torch.tensor(static_features, dtype=torch.float32)
        self.historical_data = torch.tensor(historical_data, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.float32)

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        # Return the idea (text), static features, historical data, and the target
        return self.ideas[idx], self.static_features[idx], self.historical_data[idx], self.targets[idx]


train_dataset = StockDataset(ideas=ideas_train,
                             static_features=static_train,
                             historical_data=hist_train,
                             targets=y_train)

val_dataset = StockDataset(ideas=ideas_val,
                           static_features=static_val,
                           historical_data=hist_val,
                           targets=y_val)


### Training the model
Here we import the model and set it up for training

In [4]:
import torch.optim as optim
import torch.nn as nn
import Model.HybridStockPredictionModel
from Model.HybridStockPredictionModel import StockPerformancePredictionModel

# Model initialization
static_feature_dim = static_features.shape[1]
historical_dim = historical_data.shape[1]
hidden_dim = 128  # Example hidden size

# Training loop
def train_model(model, train_loader, val_loader, criterion, optimizer, device, epochs=10):
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0

        # Training phase
        for ideas, static_features, historical_data, targets in train_loader:

            assert not torch.isnan(static_features).any(), "Static features contain NaN"
            assert not torch.isinf(static_features).any(), "Static features contain Inf"
            assert not torch.isnan(historical_data).any(), "Historical data contains NaN"
            assert not torch.isinf(historical_data).any(), "Historical data contains Inf"
            
            # Move to device
            static_features, historical_data, targets = (
                static_features.to(device),
                historical_data.to(device),
                targets.to(device),
            )

            # Forward pass
            predictions = model(
                idea=ideas,
                static_features=static_features,
                historical_data=historical_data,
                use_auxiliary_inputs=True,
                predict_autoregressively=False  # Default mode
            )

            # Compute loss
            loss = criterion(predictions.squeeze(), targets)
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

            train_loss += loss.item()
            
            evaluate(model, val_loader, device, criterion)

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss/len(train_loader):.4f}")



<class 'numpy.ndarray'>
Text embedding: tensor([[-1.5166e-02,  2.5676e-02,  1.6430e-02,  2.8542e-02,  6.4637e-02,
          1.0095e-02,  3.6241e-02,  1.4687e-02, -2.8164e-02, -6.2641e-02,
         -1.5888e-02,  9.1272e-03,  4.2569e-02, -4.1921e-02,  6.1957e-02,
          5.5932e-02,  7.3675e-02,  1.2315e-03, -3.7978e-02,  2.4864e-02,
          8.8292e-02, -1.2638e-02, -3.0710e-02, -1.2769e-02, -5.3606e-02,
          5.0921e-02,  5.0819e-03, -3.1426e-02, -5.1403e-02, -3.9099e-02,
          4.9547e-02, -2.9230e-02,  1.4469e-03,  1.0283e-01, -2.6039e-02,
          8.4929e-03, -1.1604e-02, -1.0734e-01,  4.3092e-02, -5.0768e-02,
          1.6119e-02, -1.0428e-01, -7.9753e-02,  5.8440e-03,  7.2337e-02,
          6.1390e-02, -3.2648e-02,  7.2152e-02,  4.0523e-02,  1.7474e-02,
         -6.8796e-02, -5.4717e-03, -1.0541e-02, -6.8191e-03, -8.8571e-02,
          4.9290e-02, -3.3493e-03, -1.7953e-02,  1.7885e-02, -5.4872e-02,
          3.3426e-02, -2.5000e-02,  9.6295e-02,  2.5815e-02,  2.8843e-02

#### We can also create a a custom loss function and an optimizer

You can also possible create custom loss functions:

In [5]:
# Suppose the last indicator is categorical
def custom_loss(predictions, targets, model, lambda_reg=0.01):
    # Mean Squared Error Loss
    mse_loss = F.mse_loss(predictions, targets)

    # L2 regularization (sum of squared weights of the model parameters)
    l2_loss = 0
    for param in model.parameters():
        l2_loss += torch.sum(param ** 2)

    # Combine the two losses
    total_loss = mse_loss + lambda_reg * l2_loss
    return total_loss


### Evaluation
After training we are going to evaluate our model:

In [6]:
def evaluate(model, val_loader, device, criterion):
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    all_predictions = []
    all_targets = []

    with torch.no_grad():  # No need to track gradients during evaluation
        for ideas_batch, static_batch, historical_batch, target_batch in val_loader:


            # Move data to device
            ideas_batch = ideas_batch.to(device)
            static_batch = static_batch.to(device)
            historical_batch = historical_batch.to(device)
            target_batch = target_batch.to(device)

            # Get predictions
            predictions = model(ideas_batch, static_features=static_batch, historical_data=historical_batch)

            # Compute loss
            loss = criterion(predictions, target_batch)
            val_loss += loss.item()

            # Collect all predictions and targets for evaluation
            all_predictions.append(predictions.cpu().numpy())
            all_targets.append(target_batch.cpu().numpy())

    # Compute average loss
    avg_val_loss = val_loss / len(val_loader)

    # Flatten the lists to make evaluation easier
    all_predictions = np.concatenate(all_predictions, axis=0)
    all_targets = np.concatenate(all_targets, axis=0)

    # Calculate evaluation metrics
    mse = mean_squared_error(all_targets, all_predictions)
    mae = mean_absolute_error(all_targets, all_predictions)
    r2 = r2_score(all_targets, all_predictions)

    print(f'Validation Loss: {avg_val_loss:.4f}')
    print(f'MSE: {mse:.4f}')
    print(f'MAE: {mae:.4f}')
    print(f'R²: {r2:.4f}')

    return avg_val_loss, mse, mae, r2


# Integration with Training Loop

In [7]:
# Run Training and Evaluation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Static: {static_feature_dim}")
print(f"Historical: {historical_dim}")
model = StockPerformancePredictionModel(static_feature_dim, historical_dim, hidden_dim, forecast_steps).to(device)

# Define batch size
batch_size = 32

# Create DataLoader for training and validation
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_model(model, train_loader, val_loader, criterion, optimizer, device, epochs=10)


Static: 3
Historical: 24


AssertionError: Static features contain NaN